In [1]:
import os
import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models import ShallowNN
from utils import load_file
from evals import evaluate,evaluate_mae_with_confidence,influence

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [3]:
client_ids = ["0_0","0_1","0_2","0_3","0_4","0_5","1_0","1_1","1_2","1_3","1_4","1_5","2_0","2_1","2_2","2_3","2_4","2_5","3_0","3_1","3_2","3_3","3_4","3_5"]

In [4]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/_fedl_global_500.pth'))

<All keys matched successfully>

## Error Bars for Federated Learning vs Isolated Training

In [ ]:
eval_list = []
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    isolated_model_path = "checkpt/isolated/batch64_client_"+str(client)+".pth"
    isolated_model =  ShallowNN(features)
    isolated_model.load_state_dict(torch.load(isolated_model_path))
    
    isolated_mae,(iso_lower_band, iso_upper_band), _  = evaluate_mae_with_confidence(isolated_model, val_loader)
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(global_model,val_loader)
    
    eval_dict = {"client_id":client, "Isolated Average MAE": round(isolated_mae, 4),
                 "Isolated MAE lower band":round(iso_lower_band,4),
                 "Isolated MAE upper band":round(iso_upper_band,4),
                 "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    eval_list.append(eval_dict)
    #print(client,eval_dict)
eval_df = pd.DataFrame.from_dict(eval_list)
eval_df["clients"] = [i for i in range(1,25)] 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
bar_width = 0.3  # Decrease the bar width
index = eval_df.index

bar1 = ax.bar(index - bar_width / 2, eval_df['Isolated Average MAE'], bar_width, yerr=[
    (eval_df["Isolated Average MAE"] - eval_df["Isolated MAE lower band"]),
    (eval_df['Isolated MAE upper band'] - eval_df["Isolated Average MAE"])
], capsize=5, label='Isolated Model MAE')

bar2 = ax.bar(index + bar_width / 2, eval_df['Federated Average MAE'], bar_width, yerr=[
    (eval_df["Federated Average MAE"] - eval_df["Federated MAE lower band"]),
    (eval_df['Federated MAE upper band'] - eval_df["Federated Average MAE"])
], capsize=5, label='Federated Model MAE')

ax.set_xlabel('Client IDs', fontdict={'fontsize': 13})
ax.set_ylabel("Mean Absolute Error for Validation", fontdict={'fontsize': 13})
ax.set_xticks(index)
ax.set_xticklabels(eval_df['clients'])
ax.legend(fontsize=15, loc="upper right")

# Adjust the xlim to decrease space at the left and right edges
ax.set_xlim(index[0] - 0.7, index[-1] + 0.7)

plt.show()

## Influence 

In [ ]:
inf_val = []
for client in client_ids:
    model = ShallowNN(features)
    model.load_state_dict(torch.load('checkpt/influence/' + str(client)+ '_fedl_global_500.pth'))
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    inf = influence(global_model,model,val_set)
    
    inf_val.append(round(inf.item(),4))
    #df.to_csv("influence2_"+str(client) + ".csv", index=False)
data = {"client id": client_ids, "inf_val": inf_val}
data = pd.DataFrame(data)
data.to_csv("insights/influence_with_pred_diff.csv" , index=False)

## Eccentricities

In [5]:
from evals import euclidean_distance, pairwise_euclidean_distance, accumulated_proximity, full_accumulated_proximity

In [6]:
full_acc_proximity = full_accumulated_proximity(client_ids,euclidean_distance)

[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [ ]:
matrix_dict = {
        key: torch.load("hessians/iso/" + str(key) + ".pth")
        for key in client_ids
    }

In [ ]:
eccentricities = []
for client in client_ids:
    client_matrix = torch.load("hessians/iso/" + str(client) + ".pth")
    acc_proximity = 0.0
    for key in matrix_dict:
        distance = accumulated_proximity(matrix_dict[key],client_matrix,euclidean_distance)
        acc_proximity += distance
    eccentricity = 2*acc_proximity/full_acc_proximity
    eccentricities.append(round(eccentricity.item(),4))
    print(client,acc_proximity, 2*acc_proximity/full_acc_proximity)
    
ecc_hessian = {"client id": client_ids, "hess_ecc": eccentricities}
ecc_hessian = pd.DataFrame(ecc_hessian)
ecc_hessian.to_csv("insights/eccentricity_with_hessian.csv" , index=False)

In [ ]:
full_acc_proximity_p = full_accumulated_proximity(client_ids,pairwise_euclidean_distance)

In [ ]:
for client in client_ids:
    client_matrix = torch.load("hessians/iso/" + str(client) + ".pth")
    acc_proximity = 0.0
    for key in matrix_dict:
        distance = accumulated_proximity(matrix_dict[key],client_matrix,pairwise_euclidean_distance)
        acc_proximity += distance
    
    print(client,acc_proximity, 2*acc_proximity/full_acc_proximity_p)

## Global Model Based Eccentricity

In [7]:
glo_matrix_dict = {
        key: {"iso" : torch.load("hessians/iso/" + str(key) + ".pth"),
             "fed" : torch.load("hessians/fed/" + str(key) + ".pth")}
        for key in client_ids
    }

In [16]:
distances = []
for client in client_ids:
    distance = accumulated_proximity(glo_matrix_dict[client]["iso"],glo_matrix_dict[client]["fed"], euclidean_distance)
    distances.append(distance)
    #print(client,acc_proximity, 2*acc_proximity/full_acc_proximity_p)

In [18]:
full_acc = sum(distances)

In [ ]:
len(distances)

In [21]:
ecc_dis = []
for i in distances:
    ecc_dis.append(round((i/full_acc).item(),4))
ecc_diss = {"client id": client_ids, "hess_dis": ecc_dis}
ecc_diss = pd.DataFrame(ecc_diss)
ecc_diss.to_csv("insights/eccentricity_with_hes_new.csv" , index=False)

In [27]:
(glo_matrix_dict["0_0"]["iso"] - glo_matrix_dict["0_0"]["fed"]).size()

torch.Size([14785, 14785])